In [7]:
%pip install pandas

^C
Note: you may need to restart the kernel to use updated packages.


     ---------------------------------------- 11.6/11.6 MB 6.6 MB/s eta 0:00:00
     -------------------------------------- 505.5/505.5 kB 7.9 MB/s eta 0:00:00
     -------------------------------------- 345.4/345.4 kB 5.3 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [40]:
#Import necessary packages

import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score,precision_recall_fscore_support,f1_score,classification_report
import warnings
warnings.filterwarnings("ignore")

In [32]:
#Read the empowerment data
data = pd.read_csv(r"E:\Ektha\Data\RIVM_labelled_corpus.csv")
data.head(2)

,threadid,postid,threadid_postid,author,timestamp,labels,content of post
0,1,1,1_1,a9beaa20-89b2-0130-daa8-5a0007c06fd5,2013-04-17T20:25:04.422+02:00,narrative,Welkom in deze discussiegroep over vermoeidhei...
1,1,1374,1_1374,3de9bcf0-8d6b-0130-dc3e-5a0007c06fd5,2013-07-01T16:13:51.979+02:00,"narrative,question",In juni 2012 is er slokdamkanker bij mij gecon...


In [33]:
#Preprocess all the text
def preprocess(text):
    text = text.lower()
    text = re.sub("\n"," ",text) #Remove all next lines
    text = re.sub(r'<[^>]+>',"",text) # remove all html markup
    text = re.sub('[^a-zèéeêëėęûüùúūôöòóõœøîïíīįìàáâäæãåçćč&@#A-ZÇĆČÉÈÊËĒĘÛÜÙÚŪÔÖÒÓŒØŌÕÎÏÍĪĮÌ0-9- \']', "", text) #Remove special characters
    text = re.sub("\?","question_mark",text) #Replace question mark as mentioned in the paper
    return text

In [34]:
# Drop any rows from the DataFrame data that contain missing values (NaN) and split the labels into a list of labels
data = data.dropna()
data["new labels"] = data["labels"].apply(lambda row:str(row).split(","))

# Remove rows where labels occur only once
value_counts = data["new labels"].value_counts()
data = data[data['new labels'].isin(value_counts[value_counts != 1].index)]

# Apply the preprocessing function to the data column
data['content of post preprocessed'] = data['content of post'].apply(preprocess)

# Convert data into numerical format
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(list(data["new labels"]))

# Vectorization of data contents
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(data['content of post preprocessed'])

In [35]:
# Splitting the dataset while maintaining the label balance
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, stratify=y, random_state=42)

# Print shapes to verify the split
print("Training set shape:", X_train.shape, y_train.shape)
print("Testing set shape:", X_test.shape, y_test.shape)

Training set shape: (4039, 32389) (4039, 5)
Testing set shape: (1010, 32389) (1010, 5)


In [36]:
#Multi label classifier (The value C=1.0 is as per the empowerment paper)
clf = OneVsRestClassifier(LinearSVC(C=1.0, max_iter=10000, random_state=42))

In [37]:
# Train the model
clf.fit(X_train, y_train)

# Predict
y_pred = clf.predict(X_test)

# Evaluate
f1_score = f1_score(y_test, y_pred, average =None)
print("f1_score:", f1_score)

f1_score: [0.         0.         0.         0.81524927 0.        ]


In [38]:
# Compute precision, recall, and F1 score for each label
precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred, average=None)

# Get the label names
label_names = mlb.classes_

# Print results for each label
for i, label in enumerate(label_names):
    print(f"Label: {label}")
    print(f"  Precision: {precision[i]}")
    print(f"  Recall: {recall[i]}")
    print(f"  F1 Score: {f1_score[i]}")
    print()

Label: emotional_support
  Precision: 0.0
  Recall: 0.0
  F1 Score: 0.0

Label: external_source
  Precision: 0.0
  Recall: 0.0
  F1 Score: 0.0

Label: informational_support
  Precision: 0.0
  Recall: 0.0
  F1 Score: 0.0

Label: narrative
  Precision: 0.6881188118811881
  Recall: 1.0
  F1 Score: 0.8152492668621701

Label: question
  Precision: 0.0
  Recall: 0.0
  F1 Score: 0.0



In [39]:
print(classification_report(y_test, y_pred,target_names=label_names))

                       precision    recall  f1-score   support

    emotional_support       0.00      0.00      0.00       170
      external_source       0.00      0.00      0.00       149
informational_support       0.00      0.00      0.00       305
            narrative       0.69      1.00      0.82       695
             question       0.00      0.00      0.00       263

            micro avg       0.69      0.44      0.54      1582
            macro avg       0.14      0.20      0.16      1582
         weighted avg       0.30      0.44      0.36      1582
          samples avg       0.69      0.50      0.56      1582



# Check for best value of c=1.0

In [41]:
c = [0.001, 0.01 , 0.1, 1.0, 10,100,1000]

for val in c:

    #Multi label classifier (The value C=1.0 is as per the empowerment paper)
    clf = OneVsRestClassifier(LinearSVC(C=val, max_iter=10000, random_state=42))
    
    # Train the model
    clf.fit(X_train, y_train)

    # Predict
    y_pred = clf.predict(X_test)

    # Evaluate
    #f1_score = f1_score(y_test, y_pred, average =None)

    print("C:",val)
   
    print(classification_report(y_test, y_pred,target_names=label_names))

C: 0.001
                       precision    recall  f1-score   support

    emotional_support       0.00      0.00      0.00       170
      external_source       0.00      0.00      0.00       149
informational_support       0.00      0.00      0.00       305
            narrative       0.69      1.00      0.82       695
             question       0.00      0.00      0.00       263

            micro avg       0.69      0.44      0.54      1582
            macro avg       0.14      0.20      0.16      1582
         weighted avg       0.30      0.44      0.36      1582
          samples avg       0.69      0.50      0.56      1582

C: 0.01
                       precision    recall  f1-score   support

    emotional_support       1.00      0.01      0.01       170
      external_source       0.00      0.00      0.00       149
informational_support       1.00      0.01      0.03       305
            narrative       0.81      1.00      0.89       695
             question       1.00  